In [42]:
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
# import pyperclip
import time
import pandas as pd
from datetime import datetime

In [2]:
start=time.time()

In [3]:
driver=Chrome()
driver.get('https://www.swimcloud.com/login/')
driver.maximize_window()

In [4]:
username= driver.find_element(By.XPATH, "//input[@id='id_username']").send_keys('')
password= driver.find_element(By.XPATH, "//input[@id='id_password']").send_keys('')
submit_btn=driver.find_element(By.XPATH, "//button[@type='submit']")
submit_btn.click()
time.sleep(2)

In [5]:
year='2022'
# use only 'Men' and 'Women'
gender='Women'
if gender =='Men':
    url=f"https://www.swimcloud.com/recruiting/rankings/{year}/M/"
else:
    url=f"https://www.swimcloud.com/recruiting/rankings/{year}/F/"

In [6]:
driver.get('https://www.swimcloud.com/recruiting/rankings/2024/M/')

In [7]:
# recruiting_btn=driver.find_element(By.XPATH, "//a[@class='c-nav__item '][5]")
# recruiting_btn.click()

In [8]:
year='2022'
# use only 'Men' and 'Women'
gender='Women'

In [9]:
# select_year_and_gender=driver.find_elements(By.XPATH, "//select[@class='form-control c-recruiting-filter']")
# # select_year
# select_year_and_gender[0].find_element(By.XPATH, "//option[text()='" + year + "']").click()
# time.sleep(2)
# #select gender
# select_year_and_gender[1].find_element(By.XPATH, "//option[text()='" + gender + "']").click()
# time.sleep(1)

In [10]:
# ranking_btn=driver.find_element(By.XPATH, "//a[@href='/recruiting/rankings/']")
# ranking_btn.click()
# time.sleep(2)

In [11]:
all_data=[]

In [12]:
# def contact_details(link):
    
#     contacts=driver.find_element(By.XPATH,"//button[@class='btn btn-primary']")
#     contacts.click()
#     time.sleep(1)
#     contacts_info=driver.find_elements(By.XPATH,"//button[@class='btn btn-icon btn-icon-plain u-ml']")
#     contacts_info[link].click()
#     return pyperclip.paste()

In [13]:
def convert_dictionary(data):
    key_value_pairs = data.split("\n")

    # Step 2: Split each key-value pair by colon ":"
    key_value_pairs = [pair.split(": ") for pair in key_value_pairs]

    # Step 3: Create an empty dictionary and add the key-value pairs
    dictionary = {}
    for pair in key_value_pairs:
        dictionary[pair[0]] = pair[1]
    return dictionary

In [14]:
def scraper(all_profiles):
    for profile in all_profiles:
        swimcloud=profile.get_attribute('href')
        data={}
        data['gender']=gender
        profile.click()
        time.sleep(.25)
        name=driver.find_element(By.XPATH, "//h1[@class='c-toolbar__title u-flex u-flex-align-items-center']")
        name=name.text.split(" ")
        data['first_name']=name[0]
        data['last_name']=' '.join(name[1:])
        
        city_state_club=driver.find_elements(By.XPATH, "//ul[@class='o-list-inline o-list-inline--dotted']/li")
        city_state=city_state_club[0].text.split(',')
        data['city']=city_state[0]
        try:
            data['state']=city_state[1]
        except:
            data['state']=None
        data['club']=city_state_club[1].text
        try:
            instagram=driver.find_element(By.XPATH, "//a[@class='btn-icon-plain']").get_attribute('href')
            data['instagram']=instagram
        except:
            data['instagram']=None
            
#         try:
#             national_rank=driver.find_element(By.XPATH, "//p[contains(@title,'rank')]")
#             national_rank_data=national_rank.find_elements(By.XPATH,"//a[@class='c-list-bar__description']")
#             data['national rank']=national_rank_data[1].text
#             data['state rank']=national_rank_data[2].text
#             data['class']=national_rank_data[-1].text
#         except:
#             national_state_rank=driver.find_elements(By.XPATH, "//a[@class='c-list-bar__description']")
#             data['national rank']=None
#             data['state rank']=national_state_rank[1].text
#             data['class']=national_rank_data[-1].text
        rank_info=driver.find_elements(By.XPATH,"//ul[@class='c-list-bar']/li")
        if(len(rank_info[1:])==3):
            data['national_rank']=rank_info[1].text.split('\n')[1]
            data['state_rank']=rank_info[2].text.split('\n')[1]
            data['class']=rank_info[-1].text.split('\n')[1]
        if(len(rank_info[1:])==2):
            if(not (rank_info[1].text.startswith('2')) and not (rank_info[-1].text.startswith('2'))):
                data['national_rank']=rank_info[1].text.split('\n')[1]
                data['class']=rank_info[-1].split('\n')[1]
            else:
                data['state_rank']=rank_info[1].text.split('\n')[1]
                data['class']=rank_info[-1].text.split('\n')[1]
        if(len(rank_info[1:])==1):
            if(rank_info[1].text.startswith('2')):
                data['state_rank']=rank_info[1].text.split('\n')[1]
            elif (rank_info[1].text.startswith('cl')):
                data['class']=rank_info[1].text.split('\n')[1]
            else:
                data['national_state']=rank_info[1].text.split('\n')[1]
            
        
#         power index
        power_index=driver.find_element(By.XPATH, "//a[@class='c-list-bar__description']")
        power_index.click()
        time.sleep(.25)
        events_name=driver.find_elements(By.XPATH,"//td[@class='u-text-truncate u-text-semi']")
        events_time=driver.find_elements(By.XPATH,"//td[@class='c-table-clean__time u-text-semi']")
        if(len(events_name)>=3):
            for i in range(3):
                event_name=f"event name {i+1}"
                event_time=f"event {i+1} time"
                
                data[event_name]=events_name[i].text
                data[event_time]=events_time[i].text
        
        else:
            for i in range(len(events_time)):
                event_name=f"event name {i+1}"
                event_time=f"event {i+1} time"
                
                data[event_name]=events_name[i].text
                data[event_time]=events_time[i].text
                
        
#         recruting
        recruiting=driver.find_elements(By.XPATH,"//nav[@class='c-nav c-nav--inline c-nav--responsive']/a")
        recruiting[-1].click()
        time.sleep(.25)
#         contact
        try:
            contacts=driver.find_element(By.XPATH,"//button[@class='btn btn-primary']")
            contacts.click()
            contacts_info=driver.find_elements(By.XPATH,"//div[@class='u-color-text']")
            for i in range(len(contacts_info)):
                if contacts_info[i].text.startswith('+'):
                    data['phone']=contacts_info[i].text
                else:
                    email='email'+str(i+1)
                    data[email]=contacts_info[i].text

        except:
            pass
        
            
        data['swimcloud']=swimcloud+'/recruiting/'
        try:
            height=driver.find_element(By.XPATH,"//div [@class='u-mb u-text-large' and contains(.,'Height') ]/span")
            data['height']=height.text.replace("\'"," ")
        except:
            data['height']=None
            
        personal_infos=driver.find_elements(By.XPATH,"//div[@class='c-card__item row']/div")
        personal_info_data=personal_infos[0].text
        personal_data=convert_dictionary(personal_info_data)
#         print(personal_data)
        for key in personal_data:
            if key in ['City','Population']:
                continue
            data[key]=personal_data[key]

#         data['country']=personal_data['Country']
#         data['median home']=personal_data['Median home']
#         data['median income']=personal_data['Median income']
        all_data.append(data)
        driver.back()
        driver.back()
        driver.back()
        
        
        
        

In [15]:
all_profiles_page1=driver.find_elements(By.XPATH, "//a[@class='u-text-semi']")

In [16]:
scraper(all_profiles_page1)

In [17]:
page2=driver.find_element(By.XPATH,"//a[@class='c-pagination__action' and contains(.,'2')]")
page2.click()
time.sleep(1)

In [18]:
all_profiles_page2=driver.find_elements(By.XPATH, "//a[@class='u-text-semi']")
scraper(all_profiles_page2)

In [19]:
page3=driver.find_element(By.XPATH,"//a[@class='c-pagination__action' and contains(.,'3')]")
page3.click()

In [20]:
all_profiles_page3=driver.find_elements(By.XPATH, "//a[@class='u-text-semi']")
scraper(all_profiles_page3)

In [21]:
driver.close()

In [22]:
end=time.time()

In [23]:
end-start

697.335263967514

In [24]:
df = pd.DataFrame(all_data)

In [51]:
dt_string = datetime.now().strftime("%d_%m_%Y_%H%M%S")
df.to_excel(dt_string+'.xlsx')

In [50]:
df[50:80]

,gender,first_name,last_name,city,state,club,instagram,national_rank,state_rank,class,...,email6,email7,email8,email9,email10,email11,email12,email13,email14,email15
50,Women,Chase,Swearingen,Westerville,OH,New Albany Aquatic Club,None,1st,51st,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,Women,Carter,Lancaster,Jacksonville,FL,Bolles School Sharks,https://instagram.com/carter_lanc,3rd,52nd,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,Women,Daniel,Li,San Marino,CA,Rose Bowl Aquatics,None,1st,52nd,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,Women,Sam,Lorenz,Thiensville,WI,Schroeder YMCA Swim Team,https://instagram.com/sam_lorenz01,1st,54th,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,Women,Jerzy,Larys,Warsaw,POL,"IUKS ,Muszelka' Warszawa",https://instagram.com/jerzy_niedzwied%C5%BA,NaN,54th,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,Women,Abram,Mueller,Madison,WI,Badger Aquatic Club (WI),None,2nd,56th,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,Women,Ross,Paterson,Durban,RSA,Action Swim Academy,None,NaN,56th,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,Women,Bryce,Rohr,Aldie,VA,Nations Capital Swim Club,None,4th,58th,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,Women,Aleksander,Sienkiewicz,Bolesławiec,POL,AZS AWF Katowice,https://m.facebook.com/home.php?refsrc=https%3...,NaN,59th,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,Women,Diego,Balbi,Lake Worth,FL,Saint Andrews Aquatics,None,4th,59th,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df['phone']

0               NaN
1               NaN
2               NaN
3      +17864274106
4               NaN
           ...     
145    +18044670207
146             NaN
147             NaN
148             NaN
149    +14694919301
Name: phone, Length: 150, dtype: object

In [ ]:
lst=['a','b','c']

In [ ]:
lst

In [ ]:
' '.join(lst[1:])

In [29]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

now = 2023-04-13 12:18:20.381047
date and time = 13/04/2023 12:18:20


In [30]:
dt_string.split('/')

['13', '04', '2023 12:18:20']

In [37]:
a=datetime.today()

In [38]:
a

datetime.datetime(2023, 4, 13, 12, 23, 5, 501889)

In [41]:
now = datetime.now() # current date and time

year = now.strftime("%Y")
print("year:", year)

month = now.strftime("%m")
print("month:", month)

day = now.strftime("%d")
print("day:", day)

time = now.strftime("%H/%M/%S")
print("time:", time)

date_time = now.strftime("%m/%d/%Y, %H%M%S")
print("date and time:",date_time)

year: 2023
month: 04
day: 13
time: 12/24/58
date and time: 04/13/2023, 122458
